# Market Based Emission 

## Overview


This notebook walks us through to Calculate emission(Market Based Emission) hotspots in the supply chain from resources and entry risks in the chain.

Scope 2 Emissions are Indirect GHG emissions that can be indirectly controlled by an organization, such as the purchase of electricity

Market Based Emisssion is part of Scope 2 Emissions.


### Setup

Make sure you have Python installed in your system (python 3+ is required).


<b>Note:</b> To run this notebook seamlessly, you have to first configure your GHG Emission APIs' credentials in a file named `secrets.ini` in the format below:

```
[EI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>



```

Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the config below.

```config.read('../../auth/secrets.ini')```

In [2]:
# Install the packages below using pip/pip3 based on your python version.
# pip install pandas,configparser,json,IPython
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

#### Get Token

Run the below code snippet to generate the Auth Bearer Token using your api_key configured in secrets.ini.

In [3]:
config = configparser.RawConfigParser()
config.read(['../../auth/secrets.ini','../../auth/config.ini'])

EI_API_KEY  = config.get('EI', 'api.api_key')
EI_CLIENT_ID = config.get('EI', 'api.client_id')
EI_AUTH_ENPOINT = config.get('EI', 'api.auth_endpoint')
EI_BASE_URL = config.get('EI', 'api.base_url')

def get_bearer_token(token):
    headers = {
        'Content-Type': 'application/json',
        'cache-control': 'no-cache',
    }
    data = {"apiKey":token, "clientId":EI_CLIENT_ID}
    bearer_token_req = requests.post(EI_AUTH_ENPOINT, headers=headers, data= json.dumps(data))
    if bearer_token_req.status_code != 200:
        print( "Error in getting Bearer token. Error code : ",bearer_token_req.status_code)
    return bearer_token_req.json()['access_token']

mybearer_token = get_bearer_token(EI_API_KEY)


The following code snippet set the Market Based Emission Parmeters


### Utility Functions

#### Function to call API

In [4]:
def Call_Carbon_API(CO2API, Co2Data, token):
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer '+token,
        'Content-Type': 'application/json',
    }

    CO2response_req = requests.post( CO2API, headers=headers, data=json.dumps(Co2Data))
    if CO2response_req.status_code != 200:
        print( "Problem! Error in generating response. Error code : ",CO2response_req.status_code)
    return CO2response_req.json()

#### Input format for API

In [5]:
def customer_id(customerid):
    Input =  {
              "id": customerid
          }
    return Input 

def customer_onBehalfOfClient(companyId,companyName):
    Input =  {
              "companyId": companyId,
              "companyName": companyName
          }
    return Input  


def customer_organisation(departmentId,departmentName):
    Input =  {
              "departmentId": departmentId,
              "departmentName": departmentName
          }
    return Input  


def customer_requestType(requestType):
    Input =  {
              "requestType": requestType,
             }
    return Input     


def customer_location(country,stateProvince,zipPostCode,city):
    Input =  {
              "country": country,
              "stateProvince": stateProvince,
              "zipPostCode": zipPostCode,
              "city": city
          }
    return Input        


def customer_site(siteId,siteName,buildingId,buildingName):
    Input = {
              "siteId": siteId,
              "siteName": siteName,
              "buildingId": buildingId,
              "buildingName": buildingName
          }               
    return Input

def market_input(commodity,energyConsumedMWh,energySupplierResidualEF,supplierNameREC,energyPurchasedMWhREC,emissionFactorREC,
                sourceREC,expiryDateREC,instrumentTypeREC,trackingSystemREC,certificateNumberREC,supplierNameREDI,
                energyPurchasedMWhREDI,emissionFactorREDI,sourceREDI,expiryDateREDI,certificateNumberREDI,
                supplierNameDEPC,energyPurchasedMWhDEPC, suppliedEmissionFactorDEPC,expiryDateDEPC,certificateNumberDEPC):
    input = {"commodity": commodity,
             "energyConsumedMWh": energyConsumedMWh,
             "energySupplierResidualEF": energySupplierResidualEF,
             "supplierNameREC": supplierNameREC,
             "energyPurchasedMWhREC":energyPurchasedMWhREC,
             "emissionFactorREC":emissionFactorREC,
             "sourceREC": sourceREC,
             "expiryDateREC": expiryDateREC,
             "instrumentTypeREC": instrumentTypeREC,
             "trackingSystemREC": trackingSystemREC,
             "certificateNumberREC": certificateNumberREC,
             "supplierNameREDI": supplierNameREDI,
             "energyPurchasedMWhREDI": energyPurchasedMWhREDI,
             "emissionFactorREDI": emissionFactorREDI,
             "sourceREDI": sourceREDI,
             "expiryDateREDI": expiryDateREDI,
             "certificateNumberREDI": certificateNumberREDI,
             "supplierNameDEPC": supplierNameDEPC,
             "energyPurchasedMWhDEPC": energyPurchasedMWhDEPC,
             "suppliedEmissionFactorDEPC": suppliedEmissionFactorDEPC,
             "expiryDateDEPC": expiryDateDEPC,
             "certificateNumberDEPC": certificateNumberDEPC
  }
    return input

### Market Based Emission Query - Example

Run the below snippet to set the request payload for Market Based Emission Endpoint Query

To check the accepted values for the individual fields of the request payload please visit:
https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API#market

In [6]:
request_payload ={'customID'        : customer_id(customerid='tesco_stationary_3838'), 
                   'onBehalfOfClient': customer_onBehalfOfClient(companyId='877889877',
                                                                 companyName='Tesco') , 
                   'organisation'    : customer_organisation(departmentId='384834',
                                                             departmentName='lloyds_retail'), 
                   'requestType'     : customer_requestType(requestType='ACTUAL'), 
                   'location'        : customer_location(country='USA',
                                                         stateProvince='Alabama',
                                                         zipPostCode='',
                                                         city=''),
                   'site'            : customer_site(siteId='T2383823',
                                                     siteName='TescoCanaryWharf',
                                                     buildingId='B38434',
                                                     buildingName='Building6'), 
                   'timePeriod'      : {'year' : '2021', 
                                        'month': '1'},
                   'activityData' : market_input(commodity='electricity',
                                                 energyConsumedMWh='1000',
                                                 energySupplierResidualEF='na',
                                                 supplierNameREC='XXX',
                                                 energyPurchasedMWhREC='20',
                                                 emissionFactorREC='0.03',
                                                 sourceREC='solar',
                                                 expiryDateREC='12-31-2020',
                                                 instrumentTypeREC='bundled',
                                                 trackingSystemREC='ERCOT',
                                                 certificateNumberREC='1168',
                                                 supplierNameREDI='Apex Clean Energy',
                                                 energyPurchasedMWhREDI='10',
                                                 emissionFactorREDI='na',
                                                 sourceREDI='wind',
                                                 expiryDateREDI='12-31-2020',
                                                 certificateNumberREDI='1078',
                                                 supplierNameDEPC='Invenergy',
                                                 energyPurchasedMWhDEPC='10', 
                                                 suppliedEmissionFactorDEPC='0.25',
                                                 expiryDateDEPC='12-31-2020',
                                                 certificateNumberDEPC='2089')
                   }        
       
print("Requests payload is :",json.dumps(request_payload, indent=2))  


Requests payload is : {
  "customID": {
    "id": "tesco_stationary_3838"
  },
  "onBehalfOfClient": {
    "companyId": "877889877",
    "companyName": "Tesco"
  },
  "organisation": {
    "departmentId": "384834",
    "departmentName": "lloyds_retail"
  },
  "requestType": {
    "requestType": "ACTUAL"
  },
  "location": {
    "country": "USA",
    "stateProvince": "Alabama",
    "zipPostCode": "",
    "city": ""
  },
  "site": {
    "siteId": "T2383823",
    "siteName": "TescoCanaryWharf",
    "buildingId": "B38434",
    "buildingName": "Building6"
  },
  "timePeriod": {
    "year": "2021",
    "month": "1"
  },
  "activityData": {
    "commodity": "electricity",
    "energyConsumedMWh": "1000",
    "energySupplierResidualEF": "na",
    "supplierNameREC": "XXX",
    "energyPurchasedMWhREC": "20",
    "emissionFactorREC": "0.03",
    "sourceREC": "solar",
    "expiryDateREC": "12-31-2020",
    "instrumentTypeREC": "bundled",
    "trackingSystemREC": "ERCOT",
    "certificateNumberREC"

Execute the below code to Run the Market Based Emission API



In [7]:
EI_API_ENDPOINT = f"{EI_BASE_URL}/v2/carbon/market"
CO2Response = Call_Carbon_API(EI_API_ENDPOINT , request_payload, mybearer_token)

df_res = pd.json_normalize(CO2Response)
df_styled = df_res.style.set_table_styles([{
    'selector': 'table',
    'props' : [
        ('display', 'block'),
        ('overflow-x', 'auto'),
        ('white-space', 'nowrap')
    ]
}])
display_summary(df_styled)

,CO2e (metric tonne),description
0,468.780000,"Total CO2e (metric tonne) avoided is 0.00 ... Invalid Renewable Energy Certificate (REC) - Expired. Invalid Renewable Energy Direct Instruments (REDI) - Expired. Invalid Direct Energy Purchase Contract (DEPC) - Expired. Location based is used due to lack of market specific residual emission factor. Latest grid average available emission factor for Alabama is from the year 2018 and it is published by US - EPA of USA, aggregated by UNFCC."


#### Output Explanation

<b>CO2e</b> -            Release of amounts of total greenhouse gases emitted (metric tonne)